# Setup

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
#input args
log_name = 'logs/LOG_001.csv'
ticks_per_second_mb = 1000
out_dir = './out'

#derived args
split_logs_dir = f'{out_dir}/split_logs/'

In [3]:
#create output folders
Path(f'{out_dir}/split_logs').mkdir(parents=True, exist_ok=True)
Path(f'{out_dir}/plots').mkdir(parents=True, exist_ok=True)

# Full Log

In [4]:
full_log = pd.read_csv(log_name, delimiter = ';')
full_log.index.rename('Log Entry', inplace = True)
full_log.head()

,Timestamp,Log Entry Type,Data
Log Entry,,,
0,1,3,"0,0,0"
1,1,4,"0,0,0"
2,4,2,1
3,11,1,"1,1,0,0,0"
4,11,1,"1,2,0,0,0,0,0,0,0"


In [5]:
#convert timestamp ticks to seconds
full_log['Time_s'] = full_log['Timestamp']/ticks_per_second_mb

In [6]:
#change entry type ids to enums
full_log['Log Entry Type'].replace({1:'SENSOR', 2:'STATE', 3:'ESTIMATOR_VAR', 4: 'CONTROLLER_OUTPUT', 5:'MOTOR', 6:'MSG'}, inplace = True)

# Sensor Log

In [7]:
#extract sensor entries
sensor_log = full_log[full_log['Log Entry Type'] == 'SENSOR'].copy()
#extract data for sensor entries
sensor_log[['Sensor Board ID', 'Sensor Type', 'Data']] = sensor_log['Data'].str.split(',', expand = True, n = 2)

#change sensor type ids to enums
sensor_log['Sensor Type'] = sensor_log['Sensor Type'].astype(int)
sensor_log['Sensor Type'].replace({1:'BARO', 2:'IMU', 3:'GPS', 4: 'BATTERY'}, inplace = True)

In [8]:
#extract individual sensor types
baro_log = sensor_log[sensor_log['Sensor Type'] == 'BARO'].copy()
imu_log = sensor_log[sensor_log['Sensor Type'] == 'IMU'].copy()
gps_log = sensor_log[sensor_log['Sensor Type'] == 'GPS'].copy()
battery_log = sensor_log[sensor_log['Sensor Type'] == 'BATTERY'].copy()

## Baro Log

In [9]:
#extract data for baro sensors
baro_log[['Pressure', 'Temperature', 'Sensor Board Timestamp']] = baro_log['Data'].str.split(',', expand = True)
baro_log.drop(columns = ['Data'], inplace = True)
#reorder columns
baro_log = baro_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type', 'Pressure', 'Temperature']]

#convert temperature to Celsius
fmt_tmp = lambda x: x / 100
baro_log['Temperature'] = baro_log['Temperature'].apply(pd.to_numeric).apply(fmt_tmp)
baro_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Pressure,Temperature
Log Entry,,,,,,,,
3,11,0.011,SENSOR,1,0,BARO,0,0.00
5,11,0.011,SENSOR,2,0,BARO,0,0.00
7,11,0.011,SENSOR,3,0,BARO,0,0.00
11,21,0.021,SENSOR,1,29203,BARO,95574,33.64
13,21,0.021,SENSOR,2,29113,BARO,95551,32.91


In [10]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
baro_log.to_csv(f'{split_logs_dir}/baro.csv')

## IMU Log

In [11]:
#generate Acc_x, Acc_y, Acc_z, Gyro_x, Gyro_y, Gyro_z
dim3 = lambda x : [x + '_' + s for s in ['x', 'y', 'z']]
imu_cols = dim3('Acc') + dim3('Gyro')

#extract data for imu sensors
imu_log[imu_cols + ['Sensor Board Timestamp']] = imu_log['Data'].str.split(',', expand = True)
imu_log.drop(columns = ['Data'], inplace = True)
#reorder columns
imu_log = imu_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type'] + imu_cols]

#scale acceleration and rotation to float

#convert to gs
fmt_acc = lambda x: x / 1024
#convert to degrees per second
fmt_gyro = lambda x: x / 65.5

imu_log[dim3('Acc')] = imu_log[dim3('Acc')].apply(pd.to_numeric).apply(fmt_acc)
imu_log[dim3('Gyro')] = imu_log[dim3('Gyro')].apply(pd.to_numeric).apply(fmt_gyro)

imu_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
Log Entry,,,,,,,,,,,,
4,11,0.011,SENSOR,1,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,11,0.011,SENSOR,2,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,11,0.011,SENSOR,3,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12,21,0.021,SENSOR,1,29194,IMU,-0.011719,0.051758,-1.027344,0.106870,-0.137405,-0.213740
15,21,0.021,SENSOR,2,29104,IMU,-0.043945,-0.019531,-0.995117,0.030534,-0.229008,-0.137405


In [12]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
imu_log.to_csv(f'{split_logs_dir}/imu.csv')

## GPS Log

In [13]:
gps_cols = ['Hour', 'Minute', 'Second', 'Latitude [deg]', 'Latitude [decimal]', 'Longitude [deg]', 'Longitude [decimal]', 'Satellite', 'Altitude', 'HDOP']
gps_log[gps_cols] = gps_log['Data'].str.split(',', expand = True)
gps_log = gps_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + gps_cols]
gps_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Hour,Minute,Second,Latitude [deg],Latitude [decimal],Longitude [deg],Longitude [decimal],Satellite,Altitude,HDOP
Log Entry,,,,,,,,,,,,,,,
454,500,0.500,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
455,502,0.502,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0
456,502,0.502,SENSOR,3,GPS,0,0,0,0,0,0,0,0,0,0
460,504,0.504,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
461,504,0.504,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0


In [14]:
gps_log.to_csv(f'{split_logs_dir}/gps.csv')

## Battery Log

In [15]:
battery_cols = ['Battery', 'Consumption', 'Current', 'Supply']
battery_log[battery_cols] = battery_log['Data'].str.split(',', expand = True)
battery_log = battery_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + battery_cols]
battery_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Battery,Consumption,Current,Supply
Log Entry,,,,,,,,,
582,226,0.226,SENSOR,1,BATTERY,619,0,0,3274
592,328,0.328,SENSOR,1,BATTERY,618,0,0,3278
602,430,0.430,SENSOR,1,BATTERY,622,0,0,3274
603,532,0.532,SENSOR,1,BATTERY,616,0,0,3277
613,634,0.634,SENSOR,1,BATTERY,617,0,0,3276


In [16]:
battery_log.to_csv(f'{split_logs_dir}/battery.csv')

# State Log

In [17]:
#IDLE = 1, AIRBRAKE_TEST, THRUSTING, COASTING, DESCENT, RECOVERY

In [18]:
#extract state change entries
state_log = full_log[full_log['Log Entry Type'] == 'STATE'].copy()
state_log.rename(columns={'Data': 'State'}, inplace = True)
state_log['State'] = state_log['State'].astype(int)
state_log['State'].replace({1:'IDLE', 2:'AIRBRAKE_TEST', 3:'THRUSTING', 4:'COASTING', 5:'DESCENT', 6:'RECOVERY'}, inplace = True)
state_log.head()

,Timestamp,Log Entry Type,State,Time_s
Log Entry,,,,
2,4,STATE,IDLE,0.004
14,17,STATE,IDLE,0.017
23,27,STATE,IDLE,0.027
32,37,STATE,IDLE,0.037
41,46,STATE,IDLE,0.046


In [19]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
state_log.to_csv(f'{split_logs_dir}/state_transitions.csv')

# Estimator Variable Log

In [22]:
#extract estimator var entries
estimator_var_log = full_log[full_log['Log Entry Type'] == 'ESTIMATOR_VAR'].copy()
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log['Data'].str.split(',', expand = True)

estimator_var_log.drop(columns = ['Data'], inplace = True)
#reorder columns
estimator_var_log = estimator_var_log[['Timestamp', 'Time_s', 'Log Entry Type', 'World Position (z)', 'Rocket Velocity (z)', 'Rocket Acceleration (z)']]

fmt_est = lambda x: x / 1000
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']].apply(pd.to_numeric).apply(fmt_est)
estimator_var_log.head()

,Timestamp,Time_s,Log Entry Type,World Position (z),Rocket Velocity (z),Rocket Acceleration (z)
Log Entry,,,,,,
0,1,0.001,ESTIMATOR_VAR,0.0,0.0,0.0
9,13,0.013,ESTIMATOR_VAR,0.0,0.0,0.0
18,23,0.023,ESTIMATOR_VAR,0.0,0.0,0.0
27,33,0.033,ESTIMATOR_VAR,0.0,0.0,0.0
36,43,0.043,ESTIMATOR_VAR,0.0,0.0,0.0


In [23]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
estimator_var_log.to_csv(f'{split_logs_dir}/estimator_var.csv')

# Controller Output Log

In [24]:
#extract estimator var entries
controller_out_log = full_log[full_log['Log Entry Type'] == 'CONTROLLER_OUTPUT'].copy()
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log['Data'].str.split(',', expand = True)

controller_out_log.drop(columns = ['Data'], inplace = True)
#reorder columns
controller_out_log = controller_out_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Controller Output', 'Reference Error', 'Integrated Error']]

fmt_cnt = lambda x: x / 1000
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']].apply(pd.to_numeric).apply(fmt_cnt)
controller_out_log.head()

,Timestamp,Time_s,Log Entry Type,Controller Output,Reference Error,Integrated Error
Log Entry,,,,,,
1,1,0.001,CONTROLLER_OUTPUT,0.0,0.0,0.0
10,13,0.013,CONTROLLER_OUTPUT,0.0,0.0,0.0
19,23,0.023,CONTROLLER_OUTPUT,0.0,0.0,0.0
28,33,0.033,CONTROLLER_OUTPUT,0.0,0.0,0.0
37,43,0.043,CONTROLLER_OUTPUT,0.0,0.0,0.0


In [25]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
controller_out_log.to_csv(f'{split_logs_dir}/controller_out.csv')

# Message Log

In [26]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MOTOR'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [44]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/motor.csv')

In [45]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MSG'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [46]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/messages.csv')